# Установка зависимостей

In [4]:
!pip install transformers evaluate transformers[torch] sacrebleu -q

In [1]:
import os
from pathlib import Path

# Одиночный перевод

In [29]:
length = 256
data='xquad_tr/uzn_Latn/contexts'
target_lang_tag = data.split('/')[1]
source_lang_tag = 'tur_Latn'
bash_command = 'python /root/turkic_qa/translation_and_train.py' \
              ' --model_name_or_path ychenNLP/nllb-200-3.3B-easyproject' \
              ' --tokenizer_name facebook/nllb-200-distilled-600M' \
              f' --test_file /root/turkic_qa/translation_prep_data/translation_prep_data/{data}/data.json' \
              f' --max_source_length {length}' \
              f' --max_target_length {length}' \
              ' --do_predict' \
              ' --predict_with_generate' \
              f' --generation_max_length {length}' \
              f' --source_lang {source_lang_tag}' \
              f' --target_lang {target_lang_tag}' \
              ' --use_fast_tokenizer=True' \
              f' --output_dir /root/turkic_qa/translated_data/{data}/' \
              ' --overwrite_output_dir' \
              ' --per_device_eval_batch_size=24'
              # ' --auto_find_batch_size'

bash_command

'python /root/turkic_qa/translation_and_train.py --model_name_or_path ychenNLP/nllb-200-3.3B-easyproject --tokenizer_name facebook/nllb-200-distilled-600M --test_file /root/turkic_qa/translation_prep_data/translation_prep_data/xquad_tr/uzn_Latn/contexts/data.json --max_source_length 256 --max_target_length 256 --do_predict --predict_with_generate --generation_max_length 256 --source_lang tur_Latn --target_lang uzn_Latn --use_fast_tokenizer=True --output_dir /root/turkic_qa/translated_data/xquad_tr/uzn_Latn/contexts/ --overwrite_output_dir --per_device_eval_batch_size=24'

In [30]:
os.system(bash_command)

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-04-09 10:22:42.616127: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 10:22:42.690600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 10:22:43.680497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


04/09/2024 10:22:45 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False


/opt/conda/lib/python3.11/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


{'test': '/root/turkic_qa/translation_prep_data/translation_prep_data/xquad_tr/uzn_Latn/contexts/data.json'}


Generating test split: 1190 examples [00:00, 77475.77 examples/s]
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Running tokenizer on prediction dataset: 100%|██████████| 1190/1190 [00:00<00:00, 3506.42 examples/s]
/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataL

***** predict metrics *****
  predict_bleu               =        0.0
  predict_gen_len            =    46.9221
  predict_loss               =    10.6821
  predict_runtime            = 0:14:53.23
  predict_samples            =       6408
  predict_samples_per_second =      7.174
  predict_steps_per_second   =      0.299


0

# Потоковой перевод

In [3]:
model_name = 'ychenNLP/nllb-200-3.3B-easyproject'
tokenizer_name = 'facebook/nllb-200-distilled-600M'

script_dir = '/root/turkic_qa/translation_and_train.py'
data_dir = Path('/root/turkic_qa/translation_prep_data/')
output_dir = '/root/turkic_qa/translated_data'

batch_size = 50
num_beams = 5
length_penalty = 0.5

translated_parts = ['contexts', 'questions', 'answers']
translated_datasets = ['xquad_en', 'mlqa_val_en', 'mlqa_test_en_9000', 'squad_val_en_3500', 'squad_train_en_7500']
target_langs = ['kaz_Cyrl', 'uzn_Latn']
source_lang_tags = ['eng_Latn']
for dataset in translated_datasets:
    for source_lang_tag in source_lang_tags:
        dataset_path = data_dir / dataset
        for dirpath, dirnames, filenames in os.walk(dataset_path):
            dir_path = Path(dirpath)
            if filenames==['data.json'] and dir_path.name in translated_parts and dir_path.parent.name in target_langs:
                data_path = Path(f'{dirpath}/{filenames[0]}')

                target_lang_tag = data_path.parents[1].name
                output_path = Path(data_path.parents[2].name) / data_path.parents[1].name / data_path.parents[0].name

                if data_path.parent.name in ['contexts', 'questions']:
                    length = 256                    
                    bash_command = f'python {script_dir}' \
                                    f' --model_name_or_path {model_name}'\
                                    f' --tokenizer_name {tokenizer_name}' \
                                    f' --test_file {str(data_path)}' \
                                    f' --max_source_length {length}' \
                                    f' --max_target_length {length}' \
                                    ' --do_predict' \
                                    ' --predict_with_generate' \
                                    f' --generation_max_length {length}' \
                                    f' --source_lang {source_lang_tag}' \
                                    f' --target_lang {target_lang_tag}' \
                                    ' --use_fast_tokenizer=True' \
                                    f' --per_device_eval_batch_size={str(batch_size)}' \
                                    f' --output_dir {output_dir}/{str(output_path)}/' \
                                    ' --overwrite_output_dir'
                                    
                else:
                    length = 32
                    bash_command = f'python {script_dir}' \
                                    f' --model_name_or_path {model_name}'\
                                    f' --tokenizer_name {tokenizer_name}' \
                                    f' --test_file {str(data_path)}' \
                                    f' --max_source_length {length}' \
                                    f' --max_target_length {length}' \
                                    ' --do_predict' \
                                    ' --predict_with_generate' \
                                    f' --generation_max_length {length}' \
                                    f' --source_lang {source_lang_tag}' \
                                    f' --target_lang {target_lang_tag}' \
                                    ' --use_fast_tokenizer=True' \
                                    f' --per_device_eval_batch_size={str(batch_size)}' \
                                    f' --output_dir {output_dir}/{str(output_path)}/' \
                                    ' --overwrite_output_dir' \
                                    f' --num_beams {num_beams}' \
                                    f' --length_penalty {length_penalty}'

                # print(bash_command)
                os.system(bash_command)

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-04-26 08:40:15.967212: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 08:40:16.776733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 08:40:18.870330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


04/26/2024 08:40:23 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False


/opt/conda/lib/python3.11/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


{'test': '/root/turkic_qa/translation_prep_data/xquad_en/uzn_Latn/answers/data.json'}


Generating test split: 1190 examples [00:00, 43389.10 examples/s]
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Running tokenizer on prediction dataset: 100%|██████████| 1190/1190 [00:00<00:00, 21881.73 examples/s]
/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = Data

...

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-04-26 08:48:23.179025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 08:48:23.251560: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 08:48:24.248772: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


04/26/2024 08:48:25 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False


/opt/conda/lib/python3.11/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


{'test': '/root/turkic_qa/translation_prep_data/xquad_en/uzn_Latn/contexts/data.json'}


Generating test split: 1190 examples [00:00, 135157.24 examples/s]
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Running tokenizer on prediction dataset: 100%|██████████| 1190/1190 [00:00<00:00, 3647.05 examples/s]
/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = Data